# HINGLISH
## github.com/tushar2704

This project focuses on building a Neural Machine Translation (NMT) system to translate English sentences to Hindi. NMT has revolutionized the field of language translation by leveraging deep learning techniques to produce more accurate and natural-sounding translations.

Importing the required Libraries

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Loading data files

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [20]:
import unicodedata
import re

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeHindiString(s):
    if not isinstance(s, str):
        s = str(s)
    s = unicodeToAscii(s.strip())
    s = re.sub(r"([.!?])", r" \1", s)
    # Adjust regex patterns for Hindi text normalization
    s = re.sub(r"[^ऀ-ॿ.!?]+", r" ", s)  # Unicode range for Devanagari script
    return s

# Usage example:
hindi_text = "मेरा नाम हिन्दी में है।"
normalized_hindi_text = normalizeHindiString(hindi_text)
print(normalized_hindi_text)


मरा नाम हिनदी म ह।


In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the CSV file and extract necessary columns
    import pandas as pd
    try:
        df = pd.read_csv('/content/HINGLISH.csv', encoding='ISO-8859-1')  # Specify correct encoding
    except pd.errors.ParserError as e:
        print("Parser error:", e)
        df = pd.read_csv('/content/HINGLISH.csv', nrows=1051859 - 1)  # Skip problematic rows
    lines = df[['hindi', 'english']].values.tolist()

     # Normalize and process the data
    pairs = [[normalizeHindiString(s) for s in l] for l in lines]  # Use your Hindi normalization function

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('hindi', 'english', True)  # Adjust language names
print(random.choice(pairs))


Reading lines...
Parser error: Error tokenizing data. C error: EOF inside string starting at row 1212648
Read 1051858 sentence pairs
Trimmed to 2946 sentence pairs
Counting words...
Counted words:
english 11
hindi 3370
[' ', 'हम सीध मारग पर चला']


In [22]:
#Reading data
MAX_LENGTH = 10

# Define Hindi prefixes based on your language characteristics
hindi_prefixes = (
    "मैं ", "मैं ",
    "वह ", "वह ",
    "वह ", "वह ",
    "तुम ", "तुम ",
    "हम ", "हम ",
    "वे ", "वे "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        any(p[1].startswith(prefix) for prefix in hindi_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


In [24]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))  # Debugging line
    if not pairs:
        print("No sentence pairs remain after filtering.")  # Debugging line
        return None, None, None
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# Adjust language codes and prefix list
input_lang, output_lang, pairs = prepareData('eng', 'hin', True)

if pairs:  # Check if pairs are not None
    print(random.choice(pairs))


Reading lines...
Read 1395911 sentence pairs
Trimmed to 0 sentence pairs
No sentence pairs remain after filtering.


## The Seq2Seq Model

In [25]:
import torch
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


## The Decoder

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


## Preparing Training Data

In [28]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [29]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [30]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [32]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # This locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


In [33]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs), input_lang, output_lang)
                      for i in range(n_iters)]  # Make sure to include input_lang and output_lang
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


## Evaluation

In [34]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]


In [35]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


In [38]:
input_lang, output_lang, pairs = prepareData('hindi', 'english', True)  # Adjust language names
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder, attn_decoder, 75000, print_every=5000)


Reading lines...
Parser error: Error tokenizing data. C error: EOF inside string starting at row 1560940
Read 1051858 sentence pairs
Trimmed to 2946 sentence pairs
Counting words...
Counted words:
english 11
hindi 3370
3m 58s (- 55m 38s) (5000 6%) 4.2561
7m 57s (- 51m 40s) (10000 13%) 3.8658
11m 37s (- 46m 30s) (15000 20%) 3.6696
15m 19s (- 42m 8s) (20000 26%) 3.5021
18m 51s (- 37m 43s) (25000 33%) 3.4300
22m 35s (- 33m 53s) (30000 40%) 3.2769
26m 4s (- 29m 47s) (35000 46%) 3.2297
29m 34s (- 25m 52s) (40000 53%) 3.1651
33m 3s (- 22m 2s) (45000 60%) 3.0770
36m 33s (- 18m 16s) (50000 66%) 2.9732
40m 4s (- 14m 34s) (55000 73%) 2.9130
43m 39s (- 10m 54s) (60000 80%) 2.8531
47m 12s (- 7m 15s) (65000 86%) 2.7798
50m 59s (- 3m 38s) (70000 93%) 2.7967
54m 42s (- 0m 0s) (75000 100%) 2.8071


In [40]:
evaluateRandomly(encoder, attn_decoder)


>  
= वह उसक निरणय का पालन करगा
< हम पता चलगा  <EOS>

>  .
= वह भी शकनो पर विशवास करता था।
< वह अपनी अतरयातरा पर ह <EOS>

>  .
= हम उनह ? आवशयकताए ? कहग .
< वह अपनी अतरयातरा पर ह <EOS>

>  .
= हम इसस सहमत ह .
< वह अपनी अतरयातरा पर ह <EOS>

>  .
= वह अपन पर परिवार सहित अफरीका म बस गया।
< वह अपनी अतरयातरा जो ह ही <EOS>

>  
= वह अललाह की दहकाई हई आग ह 
< हम तो ह ह <EOS>

>  .
= वह उपातिक कामगारो क सघ का नता था।
< वह अपनी पर आग ह <EOS>

>  ? 
= वह कहगाः तम धरती म कितन वरष रह ?
< वह लोग कहग  <EOS>

>  .
= वह एक अवकाश गह म ठहरा हआ था।
< वह अपनी अतरयातरा पर ह <EOS>

>  
= हम डर हए थ 
< हम वासतव म सच <EOS>



In [39]:
%matplotlib inline

In [ ]:
output_words, attentions = evaluate(
    encoder, attn_decoder, "Season is too cold.")  # Adjust the input sentence
plt.matshow(attentions.numpy())


In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder, attn_decoder, input_sentence)  # Use your encoder and decoder instances
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

evaluateAndShowAttention("She is five years younger than me .")
evaluateAndShowAttention("She is too short .")
evaluateAndShowAttention("I'm not afraid to die .")
evaluateAndShowAttention("He is a young talented director .")
